In [1]:
#!/usr/bin/python3
### get general libraries
import copy
import os
import sys
from datetime import datetime

In [2]:
### get PDB credentials
cwd = os.getcwd()
# check where this is:
#cwd
sys.path.insert(1, cwd+"/Notebooks")
import setEnv
setEnv.SetEnv()

# import API and check authentication
import itkdb
myClient = itkdb.Client()
myClient.user.authenticate()
user=myClient.get('getUser', json={'userIdentity': myClient.user.identity})
print(user['firstName'])

Done
Kenneth


In [5]:
### start with code of component to be copied
compCode="6ede3d5f7af65cef9bb9214ccc6e3a80"
compObj=myClient.get('getComponent',json={'component':compCode})
compObj['state']

'ready'

In [6]:
# get basic template of componentType
comp_template = myClient.get('generateComponentTypeDtoSample', json={'project':compObj['project']['code'], 'code': compObj['componentType']['code'], 'requiredOnly' : False})
comp_template

{'project': 'P',
 'subproject': '...',
 'institution': '...',
 'componentType': 'SENSOR_TILE',
 'type': '...',
 'properties': {'MAN_SNO': 'BINELLYICSBIYDTDIFLDZPZCVBAWSA',
  'MAIN_VENDOR': 49,
  'SENSOR_TYPE_OR_TEST_STRUCTURE': 331,
  'VERSION': 'WLIJQKCYWGYQKTQBQGHZPEZXXJTWWD'}}

In [7]:
# make replicant by copying basic template of componentType
repObj=copy.deepcopy(comp_template)

# copy basics to replicant
basic_list=['subproject','institution','type']
for k in basic_list:
    repObj[k]=compObj[k]['code']
repObj

{'project': 'P',
 'subproject': 'PG',
 'institution': 'GL',
 'componentType': 'SENSOR_TILE',
 'type': 'RD53A-LIKE_SENSOR_TILE_150MUM',
 'properties': {'MAN_SNO': 'BINELLYICSBIYDTDIFLDZPZCVBAWSA',
  'MAIN_VENDOR': 49,
  'SENSOR_TYPE_OR_TEST_STRUCTURE': 331,
  'VERSION': 'WLIJQKCYWGYQKTQBQGHZPEZXXJTWWD'}}

In [8]:
# copy rest -- basically there is nothing else supported in registration
# avoid=['id','code','serialNumber','state','user','cts']
# # cern-itkpd-main/registerComponent/unsupportedKeys: {'type': 'warning', 'message': 'DtoIn contains unsupported keys.', 'paramMap': {'unsupportedKeyList': ['$.alternativeIdentifier', '$.stateTs', '$.stateUserIdentity', '$.dummy', '$.completed', '$.assembled', '$.trashed', '$.reworked', '$.sys', '$.currentLocation', '$.currentStage', '$.parents', '$.stages', '$.tests']}}
# unsupportedKeys= ['alternativeIdentifier', 'stateTs', 'stateUserIdentity', 'dummy', 'completed', 'assembled', 'trashed', 'reworked', 'sys', 'currentLocation', 'currentStage', 'parents', 'stages', 'tests']
# for k,v in compObj.items():
#     #print(type(v))
#     if k in repObj.keys(): continue
#     if k in unsupportedKeys: continue
#     if k in avoid: continue
#     if v==None: continue
#     if "list" in str(type(v)) and len(v)==0: continue
#     if "dict" in str(type(v)) and len(v.keys())==0: continue
#     repObj[k]=compObj[k]
# repObj

In [9]:
# copy properties to replicant
# compObj['properties']
for p in compObj['properties']:
    for rp in repObj['properties'].keys():
        if p['code']==rp:
            print(p)
            if p['value']==None: continue
            repObj['properties'][rp]=p['value']
# MAN_SNO must be unique so flag change with time
now=datetime.now()
nowStr=now.strftime("%d.%m.%Y_%H:%M")
repObj['properties']['MAN_SNO']+="_copied_"+nowStr

{'code': 'MAIN_VENDOR', 'name': 'Main Vendor', 'dataType': 'integer', 'required': True, 'default': False, 'value': 5, 'dateTime': None, 'userIdentity': None}
{'code': 'MAN_SNO', 'name': 'Manufacturer serial number', 'dataType': 'string', 'required': True, 'default': True, 'value': 'V6-3809-04', 'dateTime': None, 'userIdentity': None}
{'code': 'SENSOR_TYPE_OR_TEST_STRUCTURE', 'name': 'Sensor Type or Test Structure', 'dataType': 'integer', 'required': True, 'default': False, 'value': 3, 'dateTime': None, 'userIdentity': None}
{'code': 'VERSION', 'name': 'Version of component', 'dataType': 'string', 'required': True, 'default': False, 'value': None, 'dateTime': None, 'userIdentity': None}


In [10]:
### check what's there
repObj

{'project': 'P',
 'subproject': 'PG',
 'institution': 'GL',
 'componentType': 'SENSOR_TILE',
 'type': 'RD53A-LIKE_SENSOR_TILE_150MUM',
 'properties': {'MAN_SNO': 'V6-3809-04_copied_08.12.2020_17:20',
  'MAIN_VENDOR': 5,
  'SENSOR_TYPE_OR_TEST_STRUCTURE': 3,
  'VERSION': 'WLIJQKCYWGYQKTQBQGHZPEZXXJTWWD'}}

In [12]:
### register component
new_sensor = myClient.post('registerComponent', json=repObj)

### checking not the same!
print(compObj['serialNumber'])
print(new_sensor['component']['serialNumber'])

20UPGS85300041
20UPGS85300149


In [13]:
# a couple of useful functions
def FormatProperties(inDict):
    outDict={}
    for i in inDict:
        outDict[i['code']]=i['value']
    return outDict

def GetResult(inDict,testCode):
    #outDict={}
    for i in inDict:
        if i['code']==testCode:
            #outDict[i['code']]=i['value']
            return i['value']

In [15]:
# upload test results from original to replicant
uploadDict={}
uploadDict['component']=new_sensor['component']['code']
for t in compObj['tests']:
    print(t)
    uploadDict['testType']=t['code']
    for rt in t['testRuns']:
        if rt['state']=="requestedToDelete": continue
        uploadDict['institution']=rt['institution']['code']
        uploadDict['runNumber']=rt['runNumber']
        uploadDict['date']=rt['date']
        uploadDict['passed']=rt['passed']
        uploadDict['problems']=rt['problems']
        testObj=myClient.get('getTestRun',json={'testRun':rt['id']})
        uploadDict['properties']=FormatProperties(testObj['properties'])
        uploadDict['results']={}
        if "IV" in t['code']:
            uploadDict['results']['IV_ARRAY']=GetResult(testObj['results'],"IV_ARRAY")
            uploadDict['results']['BREAKDOWN_VOLTAGE']=GetResult(testObj['results'],"BREAKDOWN_VOLTAGE")
            uploadDict['results']['LEAK_CURRENT']=GetResult(testObj['results'],"LEAK_PLANAR")
            uploadDict['results']['IV_IMG']=GetResult(testObj['results'],"IV_IMG")
        uploadDict['comments']=testObj['comments']
        uploadDict['defects']=testObj['defects']
        updateReturn = myClient.post('uploadTestRunResults', json={**uploadDict})
        ### check return value
        print(updateReturn)

{'id': '5b1a3abf130f5600054a42a8', 'code': 'IV_MEASURE', 'name': 'IV Measurment', 'testRuns': [{'id': '5faeb70ce615fa000a1bcc0f', 'state': 'ready', 'stateTs': '2020-11-13T16:40:44.346Z', 'stateUserIdentity': None, 'runNumber': '...', 'passed': True, 'problems': False, 'date': '05.10.2020 14:35', 'institution': {'id': '59d60ee4ed67730005160cd9', 'code': 'GL', 'name': 'Glasgow University'}, 'cts': '2020-11-13T16:40:44.367Z'}, {'id': '5faeb6b4e615fa000a1bcbf2', 'state': 'requestedToDelete', 'stateTs': '2020-11-13T16:40:06.522Z', 'stateUserIdentity': '8281-1164-1', 'runNumber': '...', 'passed': True, 'problems': False, 'date': '05.10.2020 15:02', 'institution': {'id': '59d60ee4ed67730005160cd9', 'code': 'GL', 'name': 'Glasgow University'}, 'cts': '2020-11-13T16:39:16.170Z'}, {'id': '5faeac80e615fa000a1bca93', 'state': 'requestedToDelete', 'stateTs': '2020-11-13T16:38:50.115Z', 'stateUserIdentity': '8281-1164-1', 'runNumber': '...', 'passed': True, 'problems': False, 'date': '05.10.2020 14:

In [16]:
### checking new component in PDB
newObj=myClient.get('getComponent',json={'component':new_sensor['component']['code']})
newObj['tests']

[{'id': '5b1a3abf130f5600054a42a8',
  'code': 'IV_MEASURE',
  'name': 'IV Measurment',
  'testRuns': [{'id': '5fcfb6a367b73a000ab90e05',
    'state': 'ready',
    'stateTs': '2020-12-08T17:23:47.905Z',
    'stateUserIdentity': None,
    'runNumber': '...',
    'passed': True,
    'problems': False,
    'date': '05.10.2020 14:35',
    'institution': {'id': '59d60ee4ed67730005160cd9',
     'code': 'GL',
     'name': 'Glasgow University'},
    'cts': '2020-12-08T17:23:47.914Z'}]}]

In [19]:
### delete component if required
myClient.post('deleteComponent', json={'component':new_sensor['component']['code']})

### component is flagged for deletion
checkObj=myClient.get('getComponent',json={'component':new_sensor['component']['code']})
print(checkObj['state'])


requestedToDelete
